In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [5]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
data

[    Symbol             Security SEC filings             GICS Sector  \
 0      MMM                   3M     reports             Industrials   
 1      ABT  Abbott Laboratories     reports             Health Care   
 2     ABBV               AbbVie     reports             Health Care   
 3     ABMD              Abiomed     reports             Health Care   
 4      ACN            Accenture     reports  Information Technology   
 ..     ...                  ...         ...                     ...   
 500    YUM          Yum! Brands     reports  Consumer Discretionary   
 501   ZBRA   Zebra Technologies     reports  Information Technology   
 502    ZBH        Zimmer Biomet     reports             Health Care   
 503   ZION        Zions Bancorp     reports              Financials   
 504    ZTS               Zoetis     reports             Health Care   
 
                       GICS Sub-Industry    Headquarters Location  \
 0              Industrial Conglomerates    Saint Paul, Minnesota

In [7]:
print (data[0])

    Symbol             Security SEC filings             GICS Sector  \
0      MMM                   3M     reports             Industrials   
1      ABT  Abbott Laboratories     reports             Health Care   
2     ABBV               AbbVie     reports             Health Care   
3     ABMD              Abiomed     reports             Health Care   
4      ACN            Accenture     reports  Information Technology   
..     ...                  ...         ...                     ...   
500    YUM          Yum! Brands     reports  Consumer Discretionary   
501   ZBRA   Zebra Technologies     reports  Information Technology   
502    ZBH        Zimmer Biomet     reports             Health Care   
503   ZION        Zions Bancorp     reports              Financials   
504    ZTS               Zoetis     reports             Health Care   

                      GICS Sub-Industry    Headquarters Location  \
0              Industrial Conglomerates    Saint Paul, Minnesota   
1          

In [3]:
# Get current S&P table and set header column
sp500 = data[0].loc[1:,[0,1,6,7]]
columns = ['added_ticker', 'name', 'date', 'cik']
sp500.columns = columns
sp500.loc[sp500['date'].isnull(), 'date'] = '1957-01-01'

# One date is in the wrong format. Correcting it.
sp500.loc[~sp500['date'].str.match('\d{4}-\d{2}-\d{2}'), 'date'] = '1985-01-01'
sp500.loc[:,'date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
sp500 = pd.melt(sp500, id_vars=['date', 'name', 'cik'], value_vars=['added_ticker'])
sp500.head()

KeyError: "None of [Int64Index([0, 1, 6, 7], dtype='int64')] are in the [columns]"

In [68]:
sp500_adjustments = data[1]
sp500_adjustments = sp500_adjustments[2:].copy()
columns = ['date', 'added_ticker', 'added_name', 'removed_ticker', 'removed_name', 'reason']
sp500_adjustments.columns = columns
updates = sp500_adjustments[~sp500_adjustments['date'].str.contains(',')].T.shift(1).T
sp500_adjustments['date'].loc[~sp500_adjustments['date'].str.contains(',')] = np.nan
sp500_adjustments[sp500_adjustments['added_ticker'].isnull()]
sp500_adjustments.update(updates)
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_additions = sp500_adjustments[~sp500_adjustments['added_ticker'].isnull()]
sp500_additions = sp500_additions[['date', 'added_ticker', 'added_name']]
sp500_additions.rename(columns={'added_name': 'name'}, inplace=True)
sp500_additions = pd.melt(sp500_additions, id_vars=['date','name'], value_vars=['added_ticker'])
sp500_deletions = sp500_adjustments[~sp500_adjustments['removed_ticker'].isnull()]
sp500_deletions = sp500_deletions[['date', 'removed_ticker', 'removed_name']]
sp500_deletions.rename(columns={'removed_name': 'name'}, inplace=True)
sp500_deletions = pd.melt(sp500_deletions, id_vars=['date','name'], value_vars=['removed_ticker'])

sp500_history = pd.concat([sp500_deletions, sp500_additions])
sp500_history.head()

,date,name,variable,value
0,"August 9, 2019",Anadarko Petroleum,removed_ticker,APC
1,"August 9, 2019",Foot Locker,removed_ticker,FL
2,"July 15, 2019",Red Hat,removed_ticker,RHT
3,"July 1, 2019",L3 Technologies,removed_ticker,LLL
4,"June 7, 2019",Mattel Inc,removed_ticker,MAT


In [69]:
df = pd.concat([sp500, sp500_history], ignore_index=True)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='cik', ascending=False, inplace=True)
deduped_df = df[~df.duplicated(['date', 'variable', 'value'])].copy()
deduped_df.sort_values(by='date',inplace=True)
deduped_df.to_csv("sp500_history.csv")
deduped_df.head()

,cik,date,name,value,variable
183,0000072741,1957-01-01,Eversource Energy,ES,added_ticker
228,0000874766,1957-01-01,Hartford Financial Svc.Gp.,HIG,added_ticker
435,0001113169,1957-01-01,T. Rowe Price Group,TROW,added_ticker
349,0001111711,1957-01-01,NiSource Inc.,NI,added_ticker
185,0001109357,1957-01-01,Exelon Corp.,EXC,added_ticker


In [70]:
deduped_df.sort_values(by='cik', ascending=False, inplace=True)
deduped_df = deduped_df[~deduped_df.duplicated('value')]
# discovery has 2 share classes listed
deduped_df = deduped_df[~deduped_df.duplicated('cik')]
deduped_df.sort_values(by='value', inplace=True)
deduped_df.drop(['date', 'variable'], axis=1, inplace=True)
deduped_df.rename(columns={'value':'ticker'}, inplace=True)
deduped_df.to_csv("sp500_constituents.csv")
deduped_df.head()

,cik,name,ticker
12,0001090872,Agilent Technologies Inc,A
31,0000006201,American Airlines Group,AAL
8,0001158449,Advance Auto Parts,AAP
49,0000320193,Apple Inc.,AAPL
2,0001551152,AbbVie Inc.,ABBV
